In [1]:
import os
import settings
import pandas as pd

#在字典中，我们将跟踪id性能数据中出现的次数以及是否foreclosure_date曾经出现None。这将给我们foreclosure_status和performance_count。

def count_performance_rows():
    counts={}
    with open(os.path.join(settings.PROCESSED_DIR,"Performance.txt"),'r') as f:
        for i,line in enumerate(f):
            if i==0:
                #Skip header row
                continue
            loan_id,date=line.split("|")
            loan_id=int(loan_id)
            if loan_id not in counts:
                counts[loan_id]={
                    "foreclosure_status":False,
                    "performance_count":0
                }
            counts[loan_id]["performance_count"]+=1
            if len(date.strip())>0:
                counts[loan_id]["foreclosure_status"]=True
        return counts
    

创建counts字典后，如果传入a loan_id和a ，则可以创建一个从字典中提取值的函数key：  

下面的函数将从counts字典中返回适当的值，并使我们能够为Acquisition数据中的每一行分配一个foreclosure_status值和一个performance_count值。如果未找到键，则字典上的get方法将返回默认值，因此，如果在counts字典中未找到键，这将使我们能够返回合理的默认值。

In [2]:
def get_performance_summary_value(loan_id,key,counts):
    value=counts.get(loan_id,{
        "foreclosure_status":False,
        "performance_count":0
    })
    return value[key]

我们需要将采集数据转换为可在机器学习算法中使用的训练数据集。这涉及几件事：  
- 将所有列转换为数值。  
- 填写任何缺少的值。  
- 为每行 分配a performance_count和a foreclosure_status。  
- 删除任何没有很多性能历史记录（performance_count低）的行。  
几列是字符串，对于机器学习算法没有用。但是，它们实际上是类别变量，其中有一些不同的类别代码，例如R，S等等。我们可以通过为每个类别标签分配一个数字来将这些列转换为数字。  
以这种方式转换列将使我们能够在机器学习算法中使用它们。一些列还包含日期（first_payment_date和origination_date）。我们可以将这些日期分别分成几2列：  

id  date  
10  2/2015   

id month year  
10 2    2015

In [3]:
def annotate(acquisition,counts):
    acquisition["foreclosure_status"]=acquisition["id"].apply(lambda x:get_performance_summary_value(x,"foreclosure_status",counts))
    acquisition["performance_count"] = acquisition["id"].apply(lambda x: get_performance_summary_value(x, "performance_count", counts))
    for column in[
        "channel",
        "seller",
        "first_time_homebuyer",
        "loan_purpose",
        "property_type",
        "occupancy_status",
        "property_state",
        "product_type"
    ]:
        actuisition[column]=acquisition[column].astype('category').cat.codes
        
    for start in ["first_payment","origination"]:
        column="{}_date".format(start)
        acquisition["{}_year".format(start)]=pd.to_numeric(acquisition[column].str.split('/').str.get(1))
        acquisition["{}_month".format(start)] = pd.to_numeric(acquisition[column].str.split('/').str.get(0))
        del acquisition[column]
        
    acquisition=acquisition.fillna(-1)
    acquisition=acquisition[acquisition["performance_count"]>settings.MIINIMUM_TRACKING_QUARTERS]
    return acquisition

定义一个函数以读取采集数据。

In [4]:
def read():
    acquisition=pd.read_csv(os.path.join(settings.PROCESSED_DIR,"Acquisition.txt"),sep="|")
    return acquisition

定义一个函数将处理后的数据写入 processed/train.csv

In [5]:
def write(acquisition):
    acquisition.to_csv(os.path.join(settings.PROCESSED_DIR,"train.csv"),index=False)
    

如果从命令行调用此文件，例如python annotate.py：  
- 读取采集数据。  
- 计算效果数据的计数，然后将其分配给counts。  
- 注释数据acquisition框。  
- 将acquisitionDataFrame写入train.csv。

In [6]:
if __name__ =="__main__":
    acquisition=read()
    counts=count_performance_rows()
    acquisition=annotate(acquisition,counts)
    write(acquisition)

FileNotFoundError: [Errno 2] File b'processed\\Acquisition.txt' does not exist: b'processed\\Acquisition.txt'